In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
%cd '/content/drive/MyDrive/Colab Notebooks/DeepFake Detector'

/content/drive/MyDrive/Colab Notebooks/DeepFake Detector


In [3]:
from __future__ import division

import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from numpy.random import seed

import tensorflow as tf

import keras
from keras import preprocessing
from keras.preprocessing.image import ImageDataGenerator
from keras import layers, Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.optimizers import *
from keras.applications import *
from keras import metrics
from keras.losses import BinaryCrossentropy
from keras import backend as K

!pip install -U keras-tuner
from kerastuner.tuners import RandomSearch, Hyperband
from kerastuner.engine.hypermodel import HyperModel
from kerastuner.engine.hyperparameters import HyperParameters
from kerastuner import Objective

Requirement already up-to-date: keras-tuner in /usr/local/lib/python3.7/dist-packages (1.0.2)


In [4]:
%tensorflow_version 2.x
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Found GPU at: /device:GPU:0
Num GPUs Available:  1


In [5]:
IMG_HEIGHT, IMG_WIDTH = 128, 128
EPOCHS = 30
DATA_GENERATOR_SEED = 1337
BACTH_SIZE = 128
VALIDATION_SPLIT = 0
LEARNING_RATE = 1e-3

tf.random.set_seed(DATA_GENERATOR_SEED)
seed(DATA_GENERATOR_SEED)

DF_TYPE = 'diff'
LOSS = BinaryCrossentropy()
PRE_TRAINED_MODEL = 'Xception'
OPTIMIZERZ = 'SGD'

TRAIN_VAL_DIR = f'./Celeb-DF-v2/Celeb-{DF_TYPE}'
if 'bal' in TRAIN_VAL_DIR:
    TEST_DIR = TRAIN_VAL_DIR.replace('bal', 'test')
else:
    TEST_DIR = f'{TRAIN_VAL_DIR}-test'

In [6]:
TRAIN_DATAGEN = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 20,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.15,
                                   zoom_range = 0.15,
                                   horizontal_flip = True,
                                   fill_mode='nearest',
                                   validation_split = VALIDATION_SPLIT)

VAL_DATAGEN = ImageDataGenerator(rescale = 1.0/255., 
                                 validation_split = VALIDATION_SPLIT)

TEST_DATAGEN = ImageDataGenerator(rescale = 1.0/255.)

TRAIN_GENERATOR = TRAIN_DATAGEN.flow_from_directory(directory = TRAIN_VAL_DIR,
                                                    batch_size = BACTH_SIZE,
                                                    class_mode = 'binary', 
                                                    target_size = (IMG_HEIGHT, IMG_WIDTH),
                                                    subset = 'training',
                                                    seed = DATA_GENERATOR_SEED)

VALIDATION_GENERATOR = TRAIN_DATAGEN.flow_from_directory(directory = TRAIN_VAL_DIR,
                                                         batch_size = BACTH_SIZE,
                                                         class_mode = 'binary', 
                                                         target_size = (IMG_HEIGHT, IMG_WIDTH),
                                                         subset = 'validation',
                                                         seed = DATA_GENERATOR_SEED)

TEST_GENERATOR = TEST_DATAGEN.flow_from_directory(directory = TEST_DIR,
                                                  batch_size = BACTH_SIZE,
                                                  class_mode = 'binary', 
                                                  target_size = (IMG_HEIGHT, IMG_WIDTH),                                
                                                  seed = DATA_GENERATOR_SEED)

Found 4577 images belonging to 2 classes.
Found 0 images belonging to 2 classes.
Found 1652 images belonging to 2 classes.


In [7]:
train_gen_list = list(TRAIN_GENERATOR.classes)
val_gen_list = list(VALIDATION_GENERATOR.classes)
test_gen_list = list(TEST_GENERATOR.classes)

train_neg, train_pos = train_gen_list.count(0), train_gen_list.count(1)
val_neg, val_pos = val_gen_list.count(0), val_gen_list.count(1)
test_neg, test_pos = test_gen_list.count(0), test_gen_list.count(1)

print(f'\nTRAIN\nReal samples = {train_neg}\nFake samples = {train_pos}')
print(f'\nVALIDATION\nReal samples = {val_neg}\nFake samples = {val_pos}')
print(f'\nTEST\nReal samples = {test_neg}\nFake samples = {test_pos}')

pos = train_pos + val_pos
neg = train_neg + val_neg
total = pos + neg

weight_for_0 = (1 / neg)*(total)/2.0 
weight_for_1 = (1 / pos)*(total)/2.0

CLASS_WEIGHT = {0: weight_for_0, 1: weight_for_1}

print(f'\nWeight for Real = {weight_for_0:.5f}')
print(f'Weight for Fake = {weight_for_1:.5f}\n')

STEP_SIZE_TRAIN = TRAIN_GENERATOR.n//TRAIN_GENERATOR.batch_size


TRAIN
Real samples = 422
Fake samples = 4155

VALIDATION
Real samples = 0
Fake samples = 0

TEST
Real samples = 168
Fake samples = 1484

Weight for Real = 5.42299
Weight for Fake = 0.55078



In [8]:
def build_model(hp):
    PRE_TRAINED_MODEL = Xception(input_shape = (IMG_HEIGHT,IMG_WIDTH,3),
                                 include_top = False,
                                 pooling = 'avg',
                                 weights = 'imagenet')
    
    for layer in PRE_TRAINED_MODEL.layers:
        layer.trainable = False

    x = layers.Flatten()(PRE_TRAINED_MODEL.output)
    for i in range(hp.Int('N_LAYERS', min_value = 0, max_value = 10)):
        x = layers.Dense(hp.Int(f'DENSE_UNITS_{i}', min_value = 1024, max_value = 3584, step = 256), activation = 'relu')(x)
        x = layers.Dropout(hp.Float(f'DROPOUT_PROB_{i}', min_value = 0, max_value = 0.35, step = 0.05))(x)
    x = layers.Dense(hp.Int(f'DENSE_UNITS_-1', min_value = 1024, max_value = 3584, step = 256), activation = 'relu')(x)
    x = layers.Dense(1, activation = 'sigmoid')(x)

    MODEL = Model(PRE_TRAINED_MODEL.input, x)

    MODEL.compile(optimizer = SGD(learning_rate = LEARNING_RATE,
                                  momentum = 0.9),
                loss = BinaryCrossentropy(label_smoothing = hp.Float(f'LABEL_SMOOTHING', min_value = 0, max_value = 0.6, step = 0.05)),
                metrics = [metrics.BinaryAccuracy(name = 'acc'),
                           metrics.AUC(name = 'auc'),
                           metrics.FalsePositives(name = 'fp')])
    
    return MODEL

In [9]:
tuner = Hyperband(build_model,
                  objective = Objective('val_auc', direction = 'max'),
                  max_epochs = EPOCHS,
                  directory = 'tuner_logs',
                  project_name = f'df_{DF_TYPE}',
                  overwrite = True)

tuner.search_space_summary()

Search space summary
Default search space size: 3
N_LAYERS (Int)
{'default': None, 'conditions': [], 'min_value': 0, 'max_value': 10, 'step': 1, 'sampling': None}
DENSE_UNITS_-1 (Int)
{'default': None, 'conditions': [], 'min_value': 1024, 'max_value': 3584, 'step': 256, 'sampling': None}
LABEL_SMOOTHING (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.6, 'step': 0.05, 'sampling': None}


In [ ]:
tuner.search(TRAIN_GENERATOR,
             validation_data = TEST_GENERATOR,
             epochs = EPOCHS,
             steps_per_epoch = TRAIN_GENERATOR.n//TRAIN_GENERATOR.batch_size,
             validation_steps = TEST_GENERATOR.n//TEST_GENERATOR.batch_size,
             class_weight = CLASS_WEIGHT,
             verbose = 1,
             callbacks = [EarlyStopping(monitor = 'val_auc',
                                        patience = EPOCHS//10,
                                        mode = 'max',
                                        verbose = 1)])


Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
N_LAYERS          |3                 |?                 
DENSE_UNITS_-1    |2048              |?                 
LABEL_SMOOTHING   |0.55              |?                 
tuner/epochs      |2                 |?                 
tuner/initial_e...|0                 |?                 
tuner/bracket     |3                 |?                 
tuner/round       |0                 |?                 

Epoch 1/2
 2/35 [>.............................] - ETA: 52:22 - loss: 0.6751 - acc: 0.5684 - auc: 0.6699 - fp: 5.0000

In [ ]:
tuner.results_summary()

In [ ]:
BEST_MODEL = tuner.get_best_models()[0]

Y_pred = BEST_MODEL.predict(TEST_GENERATOR, verbose = 1)
Y_true = TEST_GENERATOR.classes

fpr, tpr, _ = roc_curve(Y_true, Y_pred)
try:
    roc_auc = auc(fpr, tpr)
except:
    pass
    
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve (area = %0.3f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
for thrshld in map(lambda x: x/100, range(45,56)):
    y_pred = (Y_pred > thrshld).astype(int)
    print(f'\nTHRESHOLD = {thrshld}')
    print(f'\nCONFUSION MATRIX\n{confusion_matrix(Y_true, y_pred)}')
    print(f'\nCLASSIFICATION REPORT\n{classification_report(Y_true, y_pred, target_names = ["REAL", "FAKE"])}\n\n')
    print('________________________________________________________________\n\n')

In [ ]:
# IMG_HEIGHT, IMG_WIDTH = 299, 299
# EPOCHS = 30
# DATA_GENERATOR_SEED = 1337
# BACTH_SIZE = 256
# VALIDATION_SPLIT = 0
# LEARNING_RATE = 5e-2

# DF_TYPE = 'diff' # 'rnd', 'diff-bal', 'avg-bal', 'rnd-bal']

# TRAIN_VAL_DIR = f'./Celeb-DF-v2/Celeb-{DF_TYPE}'
# if 'bal' in TRAIN_VAL_DIR:
#     TEST_DIR = TRAIN_VAL_DIR.replace('bal', 'test')
# else:
#     TEST_DIR = f'{TRAIN_VAL_DIR}-test'

# tf.random.set_seed(DATA_GENERATOR_SEED)
# seed(DATA_GENERATOR_SEED)

# TRAIN_DATAGEN = ImageDataGenerator(rescale = 1./255.,
#                                    rotation_range = 20,
#                                    width_shift_range = 0.2,
#                                    height_shift_range = 0.2,
#                                    shear_range = 0.15,
#                                    zoom_range = 0.15,
#                                    horizontal_flip = True,
#                                    fill_mode='nearest',
#                                    validation_split = VALIDATION_SPLIT)

# VAL_DATAGEN = ImageDataGenerator(rescale = 1.0/255., 
#                                  validation_split = VALIDATION_SPLIT)

# TEST_DATAGEN = ImageDataGenerator(rescale = 1.0/255.)

# TRAIN_GENERATOR = TRAIN_DATAGEN.flow_from_directory(directory = TRAIN_VAL_DIR,
#                                                     batch_size = BACTH_SIZE,
#                                                     class_mode = 'binary', 
#                                                     target_size = (IMG_HEIGHT, IMG_WIDTH),
#                                                     subset = 'training',
#                                                     seed = DATA_GENERATOR_SEED)

# VALIDATION_GENERATOR = TRAIN_DATAGEN.flow_from_directory(directory = TRAIN_VAL_DIR,
#                                                          batch_size = BACTH_SIZE,
#                                                          class_mode = 'binary', 
#                                                          target_size = (IMG_HEIGHT, IMG_WIDTH),
#                                                          subset = 'validation',
#                                                          seed = DATA_GENERATOR_SEED)

# TEST_GENERATOR = TEST_DATAGEN.flow_from_directory(directory = TEST_DIR,
#                                                   batch_size = BACTH_SIZE,
#                                                   class_mode = 'binary', 
#                                                   target_size = (IMG_HEIGHT, IMG_WIDTH),                                
#                                                   seed = DATA_GENERATOR_SEED)
# train_gen_list = list(TRAIN_GENERATOR.classes)
# val_gen_list = list(VALIDATION_GENERATOR.classes)
# test_gen_list = list(TEST_GENERATOR.classes)

# train_neg, train_pos = train_gen_list.count(0), train_gen_list.count(1)
# val_neg, val_pos = val_gen_list.count(0), val_gen_list.count(1)
# test_neg, test_pos = test_gen_list.count(0), test_gen_list.count(1)

# print(f'\nTRAIN\nReal samples = {train_neg}\nFake samples = {train_pos}')
# print(f'\nVALIDATION\nReal samples = {val_neg}\nFake samples = {val_pos}')
# print(f'\nTEST\nReal samples = {test_neg}\nFake samples = {test_pos}')

# pos = train_pos + val_pos
# neg = train_neg + val_neg
# total = pos + neg

# weight_for_0 = (1 / neg)*(total)/2.0 
# weight_for_1 = (1 / pos)*(total)/2.0

# CLASS_WEIGHT = {0: weight_for_0, 1: weight_for_1}

# print(f'\nWeight for Real = {weight_for_0:.5f}')
# print(f'Weight for Fake = {weight_for_1:.5f}\n')

# STEP_SIZE_TRAIN = TRAIN_GENERATOR.n//TRAIN_GENERATOR.batch_size

# PRE_TRAINED_MODEL = Xception(input_shape = (IMG_HEIGHT,IMG_WIDTH,3),
#                                             include_top = False,
#                                             pooling = 'avg',
#                                             weights = 'imagenet')
# for layer in PRE_TRAINED_MODEL.layers:
#     layer.trainable = False

# x = layers.Flatten()(PRE_TRAINED_MODEL.output)
# x = layers.Dense(2048, activation = 'relu')(x)
# x = layers.Dropout(0.2)(x)
# x = layers.Dense(1024, activation = 'relu')(x)
# x = layers.Dropout(0.2)(x)
# x = layers.Dense(512, activation = 'relu')(x)
# x = layers.Dropout(0.2)(x)
# x = layers.Dense(1, activation = 'sigmoid')(x)

# MODEL = Model(PRE_TRAINED_MODEL.input, x)

# MODEL.compile(optimizer = SGD(learning_rate = LEARNING_RATE,
#                               momentum = 0.9),
#               loss = BinaryCrossentropy(),
#               metrics = [metrics.BinaryAccuracy(name = 'acc'),
#                          metrics.AUC(name = 'auc'),
#                          metrics.FalsePositives(name = 'fp')])

# HISTORY = MODEL.fit(TRAIN_GENERATOR,
#                     epochs = EPOCHS,
#                     steps_per_epoch = STEP_SIZE_TRAIN,
#                     validation_data = TEST_GENERATOR,
#                     validation_steps = TEST_GENERATOR.n//TEST_GENERATOR.batch_size,
#                     verbose = 1,
#                     class_weight = CLASS_WEIGHT,
#                     callbacks = [ReduceLROnPlateau(monitor = 'val_auc', 
#                                                    patience = 1,
#                                                    factor = 0.0001**(1/float(EPOCHS)),
#                                                    verbose = 1,
#                                                    min_lr=1e-6)])


In [ ]:
# MODEL.evaluate(TEST_GENERATOR)

# STEP_SIZE_TEST = TEST_GENERATOR.n//TEST_GENERATOR.batch_size
# TEST_GENERATOR.reset()
# preds = MODEL.predict(TEST_GENERATOR, verbose = 1)

# fpr, tpr, _ = roc_curve(TEST_GENERATOR.classes, preds)
# try:
#     roc_auc = auc(fpr, tpr)
# except:
#     pass
    
# plt.figure()
# lw = 2
# plt.plot(fpr, tpr, color='darkorange',
# lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
# plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver Operating Characteristic')
# plt.legend(loc="lower right")
# plt.show()

# acc = HISTORY.history['acc']
# auc = HISTORY.history['auc']
# loss = HISTORY.history['loss']
# fp = HISTORY.history['fp']

# val_acc = HISTORY.history['val_acc']
# val_auc = HISTORY.history['val_auc']
# val_loss = HISTORY.history['val_loss']
# val_fp = HISTORY.history['val_fp']

# epochs = range(len(acc))

# fig, axs = plt.subplots(2, 2, figsize = (10,10))

# axs[0, 0].plot(epochs, acc, 'r', label='Training Binary Accuracy')
# axs[0, 0].plot(epochs, val_acc, 'b', label='Validation Binary Accuracy')
# axs[0, 0].set_title('Training and Validation Binary Accuracy')
# axs[0, 0].legend()

# axs[0, 1].plot(epochs, loss, 'r', label='Training Loss')
# axs[0, 1].plot(epochs, val_loss, 'b', label='Validation Loss')
# axs[0, 1].set_title('Training and Validation Loss')
# axs[0, 1].legend()

# axs[1, 0].plot(epochs, auc, 'r', label='Training AUC')
# axs[1, 0].plot(epochs, val_auc, 'b', label='Validation AUC')
# axs[1, 0].set_title('Training and Validation AUROC')
# axs[1, 0].legend()

# axs[1, 1].plot(epochs, fp, 'r', label='Training False Positives')
# axs[1, 1].plot(epochs, val_fp, 'b', label='Validation False Positives')
# axs[1, 1].set_title('Training and Validation False Positives')
# axs[1, 1].legend()

In [ ]:
# for DF_TYPE in tqdm(DF_TYPEZ):
#     for LOSS in tqdm(LOSSEZ):
#         for MDL in tqdm(PRE_TRAINED_MODELZ):
#             for OPTIMIZER in tqdm(OPTIMIZERZ):
                
#                 EXPERIMENT_NAME = f'{DF_TYPE}-{MDL}-{OPTIMIZER}'.replace('-', '_').lower()

#                 try:
#                     os.mkdir(f'\n./Checkpoints/{EXPERIMENT_NAME}')
#                 except OSError:
#                     try:
#                         print(f'loading model from ./Checkpoints/{EXPERIMENT_NAME}')
#                         tf.saved_model.load(f'./Checkpoints/{EXPERIMENT_NAME}')
#                     except: 
#                         pass

#                 TRAIN_VAL_DIR = f'./Celeb-DF-v2/Celeb-{DF_TYPE}'
#                 if 'bal' in TRAIN_VAL_DIR:
#                     TEST_DIR = TRAIN_VAL_DIR.replace('bal', 'test')
#                 else:
#                     TEST_DIR = f'{TRAIN_VAL_DIR}-test'

#                 TRAIN_DATAGEN = ImageDataGenerator(rescale = 1./255.,
#                                                 rotation_range = 40,
#                                                 width_shift_range = 0.2,
#                                                 height_shift_range = 0.2,
#                                                 shear_range = 0.2,
#                                                 zoom_range = 0.2,
#                                                 horizontal_flip = True,
#                                                 validation_split = VALIDATION_SPLIT)

#                 VAL_DATAGEN = ImageDataGenerator(rescale = 1.0/255., 
#                                                 validation_split = VALIDATION_SPLIT)

#                 TEST_DATAGEN = ImageDataGenerator(rescale = 1.0/255.)

#                 TRAIN_GENERATOR = TRAIN_DATAGEN.flow_from_directory(directory = TRAIN_VAL_DIR,
#                                                                     batch_size = BACTH_SIZE,
#                                                                     class_mode = 'binary', 
#                                                                     target_size = (IMG_HEIGHT, IMG_WIDTH),
#                                                                     subset = 'training',
#                                                                     seed = DATA_GENERATOR_SEED)

#                 VALIDATION_GENERATOR = TRAIN_DATAGEN.flow_from_directory(directory = TRAIN_VAL_DIR,
#                                                                     batch_size = BACTH_SIZE,
#                                                                     class_mode = 'binary', 
#                                                                     target_size = (IMG_HEIGHT, IMG_WIDTH),
#                                                                     subset = 'validation',
#                                                                     seed = DATA_GENERATOR_SEED)

#                 TEST_GENERATOR = TEST_DATAGEN.flow_from_directory(directory = TEST_DIR,
#                                                                 batch_size = BACTH_SIZE,
#                                                                 class_mode = 'binary', 
#                                                                 target_size = (IMG_HEIGHT, IMG_WIDTH),                                
#                                                                 seed = DATA_GENERATOR_SEED)

#                 train_gen_list = list(TRAIN_GENERATOR.classes)
#                 val_gen_list = list(VALIDATION_GENERATOR.classes)
#                 test_gen_list = list(TEST_GENERATOR.classes)

#                 train_neg, train_pos = train_gen_list.count(0), train_gen_list.count(1)
#                 val_neg, val_pos = val_gen_list.count(0), val_gen_list.count(1)
#                 test_neg, test_pos = test_gen_list.count(0), test_gen_list.count(1)

#                 pos = train_pos + val_pos
#                 neg = train_neg + val_neg
#                 total = pos + neg

#                 weight_for_0 = (1 / neg)*(total)/2.0 
#                 weight_for_1 = (1 / pos)*(total)/2.0

#                 CLASS_WEIGHT = {0: weight_for_0, 1: weight_for_1}

#                 STEP_SIZE_TRAIN = TRAIN_GENERATOR.n//TRAIN_GENERATOR.batch_size
#                 # STEP_SIZE_VALID = VALIDATION_GENERATOR.n//VALIDATION_GENERATOR.batch_size

#                 EARLY_STOPPING = EarlyStopping(monitor = 'val_auc', 
#                                             patience = EPOCHS//2,
#                                             mode = 'max',
#                                             verbose = 1,
#                                             restore_best_weights = True)
                
#                 REDUCE_LR  = ReduceLROnPlateau(monitor = 'val_auc', 
#                                             patience = EPOCHS//6,
#                                             factor = 0.5,
#                                             verbose = 1,
#                                             min_lr=1e-6)
                
#                 MODEL_CHECKPOINT = ModelCheckpoint(filepath = f'./Checkpoints/{EXPERIMENT_NAME}/',
#                                                 monitor = 'val_auc',
#                                                 mode = 'max',
#                                                 verbose = 1,
#                                                 save_best_only = True)
                
#                 TENSOR_BOARD = TensorBoard(log_dir = f'./Checkpoints/{EXPERIMENT_NAME}/logs')

#                 PRE_TRAINED_MODEL = eval(MDL)(input_shape = (IMG_HEIGHT,IMG_WIDTH,3),
#                                                             include_top = False,
#                                                             weights = 'imagenet')

#                 for layer in PRE_TRAINED_MODEL.layers:
#                     layer.trainable = True

#                 x = layers.Flatten()(PRE_TRAINED_MODEL.output)  
#                 x = layers.Dense(1024, activation = 'relu')(x)
#                 x = layers.Dropout(0.1)(x)
#                 # x = layers.Dense(512, activation = 'relu')(x)
#                 # x = layers.Dropout(0.2)(x)
#                 # x = layers.Dense(256, activation = 'relu')(x)
#                 # x = layers.Dropout(0.2)(x)
#                 # x = layers.Dense(128, activation = 'relu')(x)
#                 # x = layers.Dropout(0.2)(x)   
#                 # x = layers.Dense(64, activation = 'relu')(x)
#                 # x = layers.Dropout(0.2)(x)                    
#                 x = layers.Dense(1, activation = 'sigmoid')(x)  

#                 MODEL = Model(PRE_TRAINED_MODEL.input, x) 

#                 MODEL.compile(optimizer = eval(OPTIMIZER)(learning_rate = LEARNING_RATE),
#                             loss = LOSS,
#                             metrics = [metrics.BinaryAccuracy(name = 'acc'),
#                                     metrics.AUC(name = 'auc'),
#                                     metrics.FalsePositives(name = 'fp')])

#                 HISTORY = MODEL.fit(TRAIN_GENERATOR,
#                                     epochs = EPOCHS,
#                                     steps_per_epoch = STEP_SIZE_TRAIN,
#                                     validation_data = TEST_GENERATOR,
#                                     validation_steps = TEST_GENERATOR.n//TEST_GENERATOR.batch_size,
#                                     verbose = 1,
#                                     class_weight = CLASS_WEIGHT,
#                                     callbacks = [EARLY_STOPPING,
#                                                 REDUCE_LR,
#                                                 MODEL_CHECKPOINT,
#                                                 TENSOR_BOARD])

#                 acc = HISTORY.history['acc']
#                 auc = HISTORY.history['auc']
#                 loss = HISTORY.history['loss']
#                 fp = HISTORY.history['fp']

#                 val_acc = HISTORY.history['val_acc']
#                 val_auc = HISTORY.history['val_auc']
#                 val_loss = HISTORY.history['val_loss']
#                 val_fp = HISTORY.history['val_fp']

#                 epochs = range(len(acc))

#                 fig, axs = plt.subplots(2, 2, figsize = (20,20))

#                 axs[0, 0].plot(epochs, acc, 'r', label='Training Binary Accuracy')
#                 axs[0, 0].plot(epochs, val_acc, 'b', label='Validation Binary Accuracy')
#                 axs[0, 0].set_title('Training and Validation Binary Accuracy')
#                 axs[0, 0].legend()

#                 axs[0, 1].plot(epochs, loss, 'r', label='Training Loss')
#                 axs[0, 1].plot(epochs, val_loss, 'b', label='Validation Loss')
#                 axs[0, 1].set_title('Training and Validation Loss')
#                 axs[0, 1].legend()

#                 axs[1, 0].plot(epochs, auc, 'r', label='Training AUC')
#                 axs[1, 0].plot(epochs, val_auc, 'b', label='Validation AUC')
#                 axs[1, 0].set_title('Training and Validation AUROC')
#                 axs[1, 0].legend()

#                 axs[1, 1].plot(epochs, fp, 'r', label='Training False Positives')
#                 axs[1, 1].plot(epochs, val_fp, 'b', label='Validation False Positives')
#                 axs[1, 1].set_title('Training and Validation False Positives')
#                 axs[1, 1].legend()

#                 fig.savefig(f'./Checkpoints/{EXPERIMENT_NAME}/{EXPERIMENT_NAME}_plt.png')

#                 Y_pred = MODEL.predict(TEST_GENERATOR)
#                 Y_true = TEST_GENERATOR.classes

#                 text_file = open(f'./Checkpoints/{EXPERIMENT_NAME}/{EXPERIMENT_NAME}_summary.txt', 'w')

#                 text_file.write(f'\n{EXPERIMENT_NAME}\n')
#                 text_file.write(f'\nTRAIN\nReal samples = {train_neg}\nFake samples = {train_pos}')
#                 text_file.write(f'\nVALIDATION\nReal samples = {val_neg}\nFake samples = {val_pos}')
#                 text_file.write(f'\nTEST\nReal samples = {test_neg}\nFake samples = {test_pos}')
#                 text_file.write(f'\nWeight for Real = {weight_for_0:.5f}')
#                 text_file.write(f'\nWeight for Fake = {weight_for_1:.5f}')
#                 # text_file.write(f'\nTrain step size = {STEP_SIZE_TRAIN}')
#                 # text_file.write(f'\nValidation step size = {STEP_SIZE_VALID}')
#                 text_file.write(f'\nEarly Stop after {EARLY_STOPPING.patience} epochs of patience based on {EARLY_STOPPING.monitor}')
#                 text_file.write(f'\nModel Checkpoints saved to {MODEL_CHECKPOINT.filepath} based on {MODEL_CHECKPOINT.monitor}')
#                 text_file.write(f'\nLearning-Rate will reduce by a factor of {REDUCE_LR.factor} on {REDUCE_LR.monitor} plateu after {REDUCE_LR.patience} epochs of patience\n')
#                 text_file.write(f'\nTensor Board logs saved to {TENSOR_BOARD.log_dir}')

#                 div = 100
#                 for thrshld in map(lambda x: x/div, range(0,div+1)):
#                     y_pred = (Y_pred > thrshld).astype(int)
#                     text_file.write(f'\nTHRESHOLD = {thrshld}')
#                     text_file.write(f'\nCONFUSION MATRIX\n{confusion_matrix(Y_true, y_pred)}')
#                     text_file.write(f'\nCLASSIFICATION REPORT\n{classification_report(Y_true, y_pred, target_names = ["REAL", "FAKE"])}\n\n')
#                     text_file.write('________________________________________________________________\n\n')
                    
#                 text_file.write(str(HISTORY.history).replace(', \'', ':\n\n'))
#                 text_file.close()